In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import snscrape.modules.twitter as sntwitter

**Defining tickers**

In [7]:
tickers = ["AAPL", "TSLA"]

**Twitter scraping example**

In [11]:
max_tweets = 100000
total_likes = 0

for i, tweet in enumerate(sntwitter.TwitterSearchScraper('AAPL').get_items()):
    if i > max_tweets:
        break
    total_likes = total_likes + tweet.likeCount

print(total_likes)